In [2]:
# import libraries
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# disable warnings, use w caution
import warnings
warnings.filterwarnings('ignore')

# project specific libs
import os
import matplotlib.pyplot as plt
import pathlib

In [3]:
# project specific path
path = '/Users/KevinBu/Desktop/clemente_lab/Projects/ampaim/'

In [15]:
# jobs39 is where the first PRJNA will be tested, 16S sequencing
# PRJNA317370
# we need to generate a Q2 mapping file like in OA
q2_oa = pd.read_csv(path + 'outputs/jobs39/oaq2.tsv', sep='\t')

# take columns i think we need
keep = ['BarcodeSequence','LinkerPrimerSequence']

# grab sample IDs
# df_map = q2_oa.loc[:,keep]

df_sra = pd.read_csv(path + 'outputs/jobs39/SraRunTable.txt', sep='\t')
df_sra = df_sra.rename(columns={'Run':'#SampleID'})

# insert in reverse order
for i in keep:
    df_sra.insert(1, i, ['NA' for _ in range(len(df_sra))])

df_sra.to_csv(path + 'outputs/jobs39/qiime_mapping_file.tsv', sep='\t')
df_sra.columns.values
# df_sra['Sample Name']

array(['#SampleID', 'LinkerPrimerSequence', 'BarcodeSequence',
       'Assay Type', 'AvgSpotLen', 'Bases', 'BioProject', 'BioSample',
       'BioSampleModel', 'Bytes', 'Center Name', 'collection_date',
       'Consent', 'DATASTORE filetype', 'DATASTORE provider',
       'DATASTORE region', 'Experiment', 'geo_loc_name_country',
       'geo_loc_name_country_continent', 'geo_loc_name', 'host',
       'Instrument', 'isolation_source', 'lat_lon', 'Library Name',
       'LibraryLayout', 'LibrarySelection', 'LibrarySource', 'Organism',
       'Platform', 'ReleaseDate', 'create_date', 'version', 'Sample Name',
       'SRA Study', '1-Methylhistidine', '3-Methylhistidine', 'Alanine',
       'allo-Isoleucine', 'alpha-Amino-N-butyric-acid',
       'alpha-Aminoadipic-acid', 'Anserine', 'Arginine', 'Asparagine',
       'Aspartic_Acid', 'beta-Alanine', 'beta-Aminoisobutyric-acid',
       'betaine', 'carnitine', 'Carnosine', 'choline', 'Citrulline',
       'Cystathionine_1', 'Cystathionine_2', 'Cystin

In [35]:
# jobs44
# one partition of HC n=30 has mean age 55.5 (8.80)	
# other partition of 20 disease has mean age 50.7 (10.1)
df_sra = pd.read_csv(path + 'outputs/jobs44/SraRunTable.csv')
df = df_sra[['age','sex']]
df = df.sort_values(by='age',ascending=True)

df_HC = df.iloc[0:30,:]
df_SjD = df.iloc[30:,:]

for df_ in [df_HC, df_SjD]:
    print(np.mean(df_.age),np.std(df_.age,ddof=1))

df.age.values

47.233333333333334 7.103584978039602
63.05 5.780866808068802


array([24, 36, 37, 40, 41, 42, 42, 44, 44, 45, 46, 47, 48, 48, 48, 48, 48,
       48, 49, 50, 52, 52, 53, 54, 54, 55, 55, 55, 56, 56, 56, 57, 57, 57,
       57, 59, 59, 60, 60, 61, 63, 64, 65, 65, 66, 67, 68, 71, 74, 75])

In [36]:
df_sra.columns.values

array(['Run', 'age', 'Assay Type', 'AvgSpotLen', 'Bases',
       'BIOMATERIAL_PROVIDER', 'BioProject', 'BioSample',
       'BioSampleModel', 'Bytes', 'Center Name', 'collection_data',
       'Consent', 'DATASTORE filetype', 'DATASTORE provider',
       'DATASTORE region', 'Experiment', 'Instrument', 'isolate',
       'Library Name', 'LibraryLayout', 'LibrarySelection',
       'LibrarySource', 'Organism', 'Platform', 'ReleaseDate',
       'create_date', 'version', 'Sample Name', 'sex', 'SRA Study',
       'tissue'], dtype=object)

In [99]:
def random_split_array(arr, split_ratio):
    np.random.shuffle(arr)  # Shuffle the array in place
    split_index = int(len(arr) * split_ratio)
    arr1 = arr[:split_index]
    arr2 = arr[split_index:]
    return arr1, arr2

# grab all combinations of ages
# combos = list(itertools.combinations(df.age, 20))

# Example usage:
my_array = df.age.values


for i in range(100000):
    split_ratio = 0.4
    arr1, arr2 = random_split_array(my_array, split_ratio)
    
    mu1 = np.mean(arr1)
    s1 = np.std(arr1,ddof=1)
    
    mu2 = np.mean(arr2)
    s2 = np.std(arr2,ddof=1)
    
    if 55.44 < mu2 and mu2 < 55.54:
        if 8.794 < s2 and s2 < 8.805:
            #if 50.54 < mu1 and mu1 < 50.74:
             #   if 10.0 < s1 and s1 < 10.2:
            print('found')
            print(arr1)
            print(mu1,s1)
            print(arr2)
            print(mu2,s2)

#np.testing.assert_approx_equal(mu1, 50.7, significant=2)
#np.testing.assert_approx_equal(s1, 10.1, significant=2)
#np.testing.assert_approx_equal(mu2, 55.5, significant=2)
#np.testing.assert_approx_equal(s2, 8.80, significant=3)

print('Done')


found
[36 42 41 52 48 48 37 75 46 56 57 52 24 56 55 68 59 64 50 48]
50.7 11.662355093024285
[63 42 60 53 57 61 65 74 48 48 67 55 56 57 57 60 44 54 40 49 45 71 54 59
 48 47 44 55 65 66]
55.46666666666667 8.795505644374852
found
[60 46 24 59 55 41 42 66 52 75 57 59 37 47 40 56 44 42 60 50]
50.6 11.618497049283548
[54 55 67 56 57 65 44 48 64 49 52 53 57 65 48 48 54 55 48 45 56 48 57 68
 74 61 71 63 36 48]
55.53333333333333 8.795505644374852
found
[56 71 64 61 49 42 63 56 52 67 37 40 46 24 48 45 36 53 55 48]
50.65 11.63603748795322
[42 41 47 75 66 48 57 57 59 52 68 48 44 55 44 54 60 65 56 57 60 48 55 57
 48 65 50 59 74 54]
55.5 8.799490580866522
found
[45 63 61 56 42 48 48 36 65 55 37 48 64 41 24 60 67 59 54 40]
50.65 11.63603748795322
[48 50 52 53 48 56 47 75 59 42 68 44 56 74 55 57 57 57 46 54 48 44 55 71
 57 60 52 49 65 66]
55.5 8.799490580866522
found
[55 48 46 68 71 36 48 57 42 47 48 60 37 55 44 45 24 56 60 67]
50.7 11.657841273490336
[53 61 42 49 41 52 66 59 44 75 57 63 57 55 54 57 6

In [78]:
df.label.value_counts()

label
A    30
B    20
Name: count, dtype: int64

In [15]:
# create mapping file
df = pd.read_csv(path + 'outputs/jobs45/gutonlyCRR.csv')
df = df[df['Run title'].str.contains('fecal')]
df = df.set_index('Accession')

# get diagnosis column
df['Diagnosis'] = df['Run title'].map(lambda x: x.split('sample:')[-1])

# create categorical
df = df.reset_index()
df.loc[-1] = ['categorical' for _ in range(len(df.columns))] # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 
df.iloc[0,0] = '#q2:types'
df = df.set_index('Accession')
df.index.name = '#SampleID'

# create host subject id
df['HostSubjectId'] = df['ID']

# export
df.to_csv(path + 'outputs/jobs45/qiime_mapping_file.tsv', sep='\t')
df.head()

,ID,Run title,BioProject accession,Experiment accession,Run data file type,Read filename 1,Read file1 MD5,DownLoad Read file1,Read filename 2,Read file2 MD5,...,DownLoad Index file2,Reference file name,MD5 for reference file,Assembly Name or Accession,Assembly Accession URL,other_db,accession_in_other_db,other_db_url,Diagnosis,HostSubjectId
#SampleID,,,,,,,,,,,,,,,,,,,,,
#q2:types,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,...,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical
CRR442594,2,D2:Amplicon sequence of human fecal sample:non...,PRJCA008752,CRX385340,fastq,CRR442594_f1.fq.gz (12307086 bytes),b7f3cf4ef58a2909b282c04ea9ca4291,ftp://download.big.ac.cn/gsa/CRA006415/CRR4425...,CRR442594_r2.fq.gz (11537276 bytes),2f8c75231ad1ca51500d811f4aaa5124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-pSS patient,2
CRR442598,6,D6:Amplicon sequence of human fecal sample:non...,PRJCA008752,CRX385344,fastq,CRR442598_f1.fq.gz (10916305 bytes),c4a7ee34c4b46ef345cf873e5cf2bf00,ftp://download.big.ac.cn/gsa/CRA006415/CRR4425...,CRR442598_r2.fq.gz (10506223 bytes),9efcb1b6596069129c67c2a87531d40e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-pSS patient,6
CRR442600,8,D8:Amplicon sequence of human fecal sample:pSS...,PRJCA008752,CRX385346,fastq,CRR442600_f1.fq.gz (12975507 bytes),3149e0d5588d0cea4b58c1a2706c7e43,ftp://download.big.ac.cn/gsa/CRA006415/CRR4426...,CRR442600_r2.fq.gz (12152647 bytes),038b8b7280f629bba6e6fa50eb5b78f2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pSS patients without treament,8
CRR442603,11,D11:Amplicon sequence of human fecal sample:pS...,PRJCA008752,CRX385349,fastq,CRR442603_f1.fq.gz (10712320 bytes),854db81c703df4130019c80e717ccad7,ftp://download.big.ac.cn/gsa/CRA006415/CRR4426...,CRR442603_r2.fq.gz (10213484 bytes),692bcab19cb7f412000af2869de5355f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pSS patients without treament,11
